In [1]:
pwd

'/Users/peterhaglich/Dropbox/Work/IARPA/HFC/gfc/gfc/src/notebooks'

In [2]:
import sys
sys.path.append("..")

In [3]:
import requests
import time
import datetime
from dateutil.parser import parse
import os
import json
import re
import shutil
import pandas as pd
import numpy as np

from collections import Counter
from gfcapi.gfcapi import GfcApi, GfcAdminApi

In [4]:
pwd

'/Users/peterhaglich/Dropbox/Work/IARPA/HFC/gfc/gfc/src/notebooks'

In [15]:
GFC_HOME = os.path.abspath("../..")
DATA_PATH = os.path.join(GFC_HOME, "data","gfc2_data")
SCORING_PATH = os.path.join(DATA_PATH, "scoring")
IFP_DATA_PATH = os.path.join(DATA_PATH, "IFPs")
IFP_CORPUS_PATH = os.path.join(DATA_PATH, "IFP Corpus")
AUTH_PATH = os.path.join(GFC_HOME, "src", "resources", "auth.json")
with open(AUTH_PATH, "r") as f:
    auth_dict = json.load(f)
    secret_token = auth_dict['gfc_token']
    print(secret_token)
SERVER = 'https://api.iarpagfchallenge.com'
Q_ENDPOINT = '/api/v1/questions'
url = SERVER + '/api/v1/questions' # The endpoint to retrieve questions
headers = {'Authorization':'Bearer ' + secret_token}
params = {} # More to come on this in a moment
instance='production'
new_token = "a0a4d8f1a34501875101650ee2d60a2dc9dd27c38f0e1262a57ce7e1af1743a1"
gf=GfcAdminApi(token=new_token,server=SERVER,verbose=False)
RCT_START = "2019-05-15"
LAST_THURS = "2019-07-04"
DATE_RE = re.compile("201[8-9]-[0-1][0-9]-[0-3][0-9]")
DATE_STR_FMT = "%Y-%m-%d"

a0a4d8f1a34501875101650ee2d60a2dc9dd27c38f0e1262a57ce7e1af1743a1


Load the IFP snapshot

In [8]:
# Find the IFP snapshots
ifp_files = [x for x in os.listdir(IFP_DATA_PATH) if x.endswith("json")
             and "Snapshot" in x]
# Find the latest snapshot date.
latest_date = np.max([parse(re.findall(DATE_RE, x)[0]) 
                      for x in ifp_files]).date().strftime(DATE_STR_FMT)
latest_sn_filename = "All IFP Snapshot {}.json".format(latest_date)
print(latest_sn_filename)

All IFP Snapshot 2019-07-09.json


In [9]:
with open(os.path.join(IFP_DATA_PATH, latest_sn_filename), "r", encoding="utf-8-sig") as f:
    ifps = json.load(f)
ifp_ids = [i["id"] for i in ifps]
#dq_id_dict = {i["id"]: i["question_id"] for i in ifps}
#dq_to_id_dict = {v:k for k,v in dq_id_dict.items()}
print(len(ifps), len(ifp_ids))

122 122


In [10]:
len(ifp_ids)

122

In [11]:
DAY_0 = "2019-05-15"
DAY_N = "2019-07-08"
rct_dr = pd.date_range(DAY_0, DAY_N)
the_days = [x.date().isoformat() for x in rct_dr]

In [16]:
for dd in the_days:
    print(dd)
    _created_after = "{0}T00:00:00.0".format(dd)
    _created_before = "{0}T23:59:59.9".format(dd)
    params = {"created_after": parse(_created_after),
              "created_before": parse(_created_before)}
    n_tries = 0
    h_success = False
    while n_tries <= 10 and h_success == False:
        try:
            _scores = gf.get_scores(**params)
            if _scores is None:
                print("None scores on {}".format(dd))
            elif len(_scores) == 0:
                print("No scores on {}".format(dd))
            else:
                print("There were {} scores on {}".format(len(_scores), dd))
                _path = os.path.join(SCORING_PATH,
                         "Scoring_Snapshot_{0}.json".format(dd))
                with open(_path, "w") as f:
                    json.dump(_scores, f, ensure_ascii=False, sort_keys=True, indent=2)
            h_success = True
        except (ValueError, AttributeError):
            time.sleep(60)
            n_tries += 1



2019-05-15
There are 0 pages
No scores on 2019-05-15
2019-05-16
There are 0 pages
No scores on 2019-05-16
2019-05-17
There are 0 pages
No scores on 2019-05-17
2019-05-18
There are 0 pages
No scores on 2019-05-18
2019-05-19
There are 0 pages
No scores on 2019-05-19
2019-05-20
There are 2 pages
There were 80 scores on 2019-05-20
2019-05-21


KeyboardInterrupt: 

In [17]:
for q_id in ifp_ids:
    _created_after = "{0}T00:00:00".format(DAY_0)
    _created_before = "{0}T23:59:59".format(DAY_N)
    params = {"created_after": parse(_created_after),
              "created_before": parse(_created_before),
              "score_type": "question",
              "scoreable_id": q_id}
    q_scores = gf.get_scores(**params)
    if q_scores is None:
        print("No scores for {}".format(q_id))
    else:
        _path = os.path.join(SCORING_PATH, "by_question",
                             "Scoring_{}.json".format(q_id))
        with open(_path, "w") as f:
            json.dump(q_scores, f, ensure_ascii=False, sort_keys=True, indent=2)

There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 19 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 19 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There are 0 pages
There ar